<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/2_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 使用matplotlib.pyplot

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

def parse_color(rgb_string):
    # 將RGB字符串轉換為三個浮點數組成的元組
    rgb = rgb_string.strip('rgb()').split(',')
    return tuple(int(x)/255 for x in rgb)

# 計算橢圓的中心、半長軸和半短軸
center_x = np.mean(x)
center_y = np.mean(y)
semi_major_axis = (max(x) - min(x)) / 2
semi_minor_axis = (max(y) - min(y)) / 2

# 繪製橢圓外框
ellipse = Ellipse((center_x, center_y), width=semi_major_axis*2, height=semi_minor_axis*2,
                  edgecolor=parse_color('rgb(50,160,160)'), fc='None', lw=2)
plt.gca().add_patch(ellipse)


# 使用zip將x, y, 和colors列表組合在一起
for x_val, y_val, color_str in zip(x, y, colors):
    plt.plot(x_val, y_val, 'o', color=parse_color(color_str), markersize=2)

'''
# 將x和y座標組合成一個點集
points = np.array(list(zip(x, y)))
# 計算點集的凸包
hull = ConvexHull(points)

# 繪製凸包的外框
for simplex in hull.simplices:
    plt.plot(points[simplex, 0], points[simplex, 1], 'k-') # k- : 繪製一條黑色的實線
    #plt.plot(points[simplex, 0], points[simplex, 1], color='blue', linewidth=2)
'''


plt.axis('equal')  # 確保x和y軸的比例相同，從而使圓形看起來是正確的
plt.show()


## 邊緣框

* 自己的

In [ ]:
# 找邊緣點 : 找出每個x下的 min/max y (反之亦然)
def find_edge_points(x, y):
    coordinates = list(zip(x, y))
    edge_points_x = {}
    edge_points_y = {}

    for x_val, y_val in coordinates:
        edge_points_x[x_val] = {
            'min': min([y for x, y in coordinates if x == x_val]),
            'max': max([y for x, y in coordinates if x == x_val])
        }
        edge_points_y[y_val] = {
            'min': min([x for x, y in coordinates if y == y_val]),
            'max': max([x for x, y in coordinates if y == y_val])
        }

    return edge_points_x, edge_points_y


# 擴展邊緣厚度 (自行設定=4)
def extend_edge_points(edge_points_x, edge_points_y):
    extended_points = set()
    for x_val in edge_points_x:
        for i in range(1, 5):  # 扩展4个点位
            extended_points.add((edge_points_x[x_val]['min'] - i, x_val))
            extended_points.add((edge_points_x[x_val]['max'] + i, x_val))
    for y_val in edge_points_y:
        for i in range(1, 5):  # 扩展4个点位
            extended_points.add((y_val, edge_points_y[y_val]['min'] - i))
            extended_points.add((y_val, edge_points_y[y_val]['max'] + i))
    return extended_points

''' PIL
for point in extended_points:
  draw.point((points[0],points[1]), fill='rgb(50, 160, 160)')

,,,



In [ ]:
# 使用
# 假设的x, y坐标列表
x = [100, 150, 200, 150, 100, 50, 100]
y = [200, 250, 300, 150, 100, 150, 200]

# 找到边缘点
edge_points_x, edge_points_y = find_edge_points(x, y)

# 扩展边缘点
extended_points = extend_edge_points(edge_points_x, edge_points_y)

# 绘制扩展的边缘点作为外框
for point in extended_points:
    plt.plot(point[1], point[0], 'o', color="#3296A8")  # 使用matplotlib颜色代码

plt.gca().invert_yaxis()  # 根据需要调整y轴方向
plt.show()


* 回顧

In [ ]:
## 填充其(右側)邊界以達到指定的最大長度(整個右邊邊界的粗度) ##
def padding_right_margin(df, max_length):
  cols = list(df.columns)
  cols.reverse() # 將col順序倒過來
  for col in cols: # 遍歷反轉後的欄位列表
    # 檢查當前column中是否有值等於1(填充邊緣的元素)。如果是，表示找到了包含1的column (最"右邊"(外側)的非空column。)。
    if sum(df[col]==1) != 0:
      right_border_col = col # 將找到的最右邊(外側)的非空column名稱(只有"一個")，並將其存儲在right_border_col變數中。
      break
  index_ls = df[df[right_border_col]==1].index # 找到right_border_col中值為1的索引，存儲在index_ls中。
  # 計算需要填充的行數。
   # (計算right_border_col中值為1的總數減去max_length的差，然後除以8並向上取整，確保不超過指定的最大長度(邊界寬度)。)
  padding_count = np.ceil((sum(df[right_border_col]==1) - max_length)/8)
  # 在特定位置填充1。
  for c in range(1, int(padding_count)+1): #從原本邊界的右邊一行開始,到
    # 在距離最右邊的非空column c行，並且在index_ls中的相應範圍內填充。可擴展右邊邊界以達到指定的最大長度。
    df.loc[min(index_ls)+4*c : max(index_ls)-4*c, right_border_col+c] = 1
  return df

# 統整

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Ellipse

# parse_color 函数定义
def parse_color(rgb_string):
    rgb = rgb_string.strip('rgb()').split(',')
    return tuple(int(x)/255 for x in rgb)

# 假设的x, y坐标列表和颜色
x = [100, 150, 200, 150, 100, 50, 100]
y = [200, 250, 300, 150, 100, 150, 200]
colors = ['rgb(255,0,0)', 'rgb(0,255,0)', 'rgb(0,0,255)', 'rgb(255,255,0)', 'rgb(0,255,255)', 'rgb(255,0,255)', 'rgb(128,128,128)']


# 绘制点
for x_val, y_val, color_str in zip(x, y, colors):
    plt.plot(x_val, y_val, 'o', color=parse_color(color_str), markersize=2)

# 计算橢圆的中心、半长轴和半短轴
center_x = np.mean(x)
center_y = np.mean(y)
semi_major_axis = (max(x) - min(x)) / 2
semi_minor_axis = (max(y) - min(y)) / 2
# 绘制橢圆外框
ellipse = Ellipse((center_x, center_y), width=semi_major_axis*2, height=semi_minor_axis*2,
                  edgecolor=parse_color('rgb(50,160,160)'), fc='None', lw=2)
plt.gca().add_patch(ellipse)

# 找到并扩展边缘点
edge_points_x, edge_points_y = find_edge_points(x, y)
extended_points = extend_edge_points(edge_points_x, edge_points_y)

# 绘制扩展的边缘点作为外框
for point in extended_points:
    plt.plot(point[1], point[0], 'o', color=parse_color('rgb(50,160,160)'), markersize=2)

plt.gca().invert_yaxis()  # 根据需要调整y轴方向
plt.axis('equal')
plt.show()

### 補充 : go

In [ ]:
import plotly.graph_objects as go
import numpy as np

# 示例的x, y坐标列表和颜色
x = [100, 150, 200, 150, 100, 50, 100]
y = [200, 250, 300, 150, 100, 150, 200]
colors = ['rgba(255,0,0,1)', 'rgba(0,255,0,1)', 'rgba(0,0,255,1)', 'rgba(255,255,0,1)', 'rgba(0,255,255,1)', 'rgba(255,0,255,1)', 'rgba(128,128,128,1)']

# 创建Figure对象
fig = go.Figure()

# (1) 绘制点及其颜色
for x_val, y_val, color in zip(x, y, colors):
    fig.add_trace(go.Scatter(x=[x_val], y=[y_val], mode='markers', marker=dict(color=color, size=10)))

# (2) 计算橢圆的中心、半长轴和半短轴
center_x = np.mean(x)
center_y = np.mean(y)
semi_major_axis = (max(x) - min(x)) / 2
semi_minor_axis = (max(y) - min(y)) / 2

# 绘制橢圆
fig.add_shape(type="ellipse",
    xref="x", yref="y",
    x0=center_x-semi_major_axis, y0=center_y-semi_minor_axis,
    x1=center_x+semi_major_axis, y1=center_y+semi_minor_axis,
    line_color="rgba(50,160,160,1)",
)

# (3) 假设扩展的外框点位（具体逻辑需要根据需求实现）
# 为了演示，我们简单地以第一个点为例进行扩展
extended_x = [x[0] - 4, x[0] + 4]
extended_y = [y[0] - 4, y[0] + 4]
fig.add_trace(go.Scatter(x=extended_x, y=extended_y, mode='markers', marker=dict(color='rgba(50,160,160,1)', size=10)))

# 设置图表布局
fig.update_layout(showlegend=False)
fig.show()
